Quam is useful to generate a configuration file in a simpler way.
The QUA program itself is not managed by quam.

Example of usage of quam

1. Create a root QuAM object

In [134]:
from qm.qua import *
from qm import QuantumMachinesManager, SimulationConfig
from quam.components import BasicQuAM, SingleChannel
from quam.components.pulses import SquarePulse
import matplotlib.pyplot as plt
import time
from QuAM_utilities import *

machine = BasicQuAM()
machine.print_summary()  # outputs the current QuAM state

QuAM:
  channels: QuamDict Empty
  octaves: QuamDict Empty


2. Define the Single Analog Output Channels

In [135]:
 
#machine.channels['AOM_tweez'] = AOM_tweez = SingleChannel(opx_output=('con1', 1))
AOM_single_repumper = machine.channels['AOM_single_repumper'] = AOM_single_repumper = SingleChannel(opx_output=('con1', 1), intermediate_frequency=200e6)
machine.channels['AOM_double_repumper'] = AOM_double_repumper = SingleChannel(opx_output=('con1', 2), intermediate_frequency=200e6)
#machine.channels['AOM_repump'] = AOM_repump = SingleChannel(opx_output=('con1', 3))
machine.channels['AOM_single_cooler'] = AOM_single_cooler = SingleChannel(opx_output=('con1', 3), intermediate_frequency=200e6)
machine.channels['AOM_double_cooler'] = AOM_double_cooler = SingleChannel(opx_output=('con1', 4), intermediate_frequency=200e6)

machine.channels['AOM_tweez_mod'] = AOM_tweez_mod = SingleChannel(opx_output=('con1', 5), intermediate_frequency=89.161e6) # for tweezer modulation
machine.channels['AOM_tweez_spots1'] = AOM_tweez_spots1 = SingleChannel(opx_output=('con1', 6), intermediate_frequency=82.6e6) # first component for generating two spots
machine.channels['AOM_tweez_spots2'] = AOM_tweez_spots2 = SingleChannel(opx_output=('con1', 6), intermediate_frequency=67.6e6) # second component for generating two spots


3. Configure the pulses

In [136]:
length = int(50e6)

amplitude_single_cooler = 0.38
amplitude_single_repumper = 0.38

amplitude_double_cooler = 0.38
amplitude_double_repumper = 0.38

amplitude_tweez_mod = 0.05

amplitude_spot1 = 0.38
amplitude_spot2 = 0.38

high_time = 50e-3 #ns
strobo_high = 20000
strobo_low = 20000

number_of_periods = 2

AOM_single_repumper.operations["continuous_high"] = SquarePulse(length=round(high_time*1e9/4)*4, amplitude=amplitude_single_repumper)
AOM_single_cooler.operations["continuous_high"] = SquarePulse(length=round(high_time*1e9/4)*4, amplitude=amplitude_single_cooler)
AOM_double_repumper.operations["high_pulse"] = SquarePulse(length=round(high_time*1e9/4)*4, amplitude=amplitude_double_repumper)
AOM_double_cooler.operations["high_pulse"] = SquarePulse(length=round(high_time*1e9/4)*4, amplitude=amplitude_double_cooler)
#low_pulse = AOM_double_repumper.operations["low_pulse"] = SquarePulse(length=round(high_time*1e9/4)*4, amplitude=0)

AOM_tweez_mod.operations['high_pulse'] = SquarePulse(length=round(high_time*1e9/4)*4, amplitude=amplitude_tweez_mod)
AOM_tweez_mod.operations['low_pulse'] = SquarePulse(length=strobo_low, amplitude=0)
AOM_tweez_spots1.operations['high_pulse'] = SquarePulse(length=round(high_time*1e9/4)*4, amplitude=amplitude_spot1)
AOM_tweez_spots2.operations['high_pulse'] = SquarePulse(length=round(high_time*1e9/4)*4, amplitude=amplitude_spot2)


In [137]:
qua_config = machine.generate_config()

4. Write the QUA program

In [138]:
with program() as my_program:
    with infinite_loop_():
        #wait_for_trigger("AOM_modulation")
        AOM_tweez_mod.play('high_pulse')
        AOM_tweez_spots1.play('high_pulse')
        AOM_tweez_spots2.play('high_pulse')
    with infinite_loop_():
        AOM_single_repumper.play("continuous_high")
        AOM_single_cooler.play("continuous_high")
        AOM_double_repumper.play("high_pulse")
        AOM_double_cooler.play("high_pulse")
    #with infinite_loop_():


        #stroboscopic(('high_pulse','low_pulse'), 'AOM_modulation', number_of_periods)
        #wait(int(1e9/8), "AOM_modulation")
        #AOM_cool.play("zero")
    #with infinite_loop_():
    #    AOM_tweez_mod.play("high_pulse")
    #    AOM_tweez_spots1.play("high_pulse")
    #    AOM_tweez_spots2.play("high_pulse")

In [139]:
with program() as my_program2:
    with infinite_loop_():
        AOM_tweez_spots1.play('high_pulse')
        AOM_tweez_spots2.play('high_pulse')
    with infinite_loop_():
        play('high_pulse', 'AOM_tweez_mod')
        play('low_pulse', 'AOM_tweez_mod')

6. Open the machine and run the program

In [140]:
%matplotlib qt 

Simulation = False

ip_address = '130.79.148.167'
qmm = QuantumMachinesManager(host=ip_address)

if Simulation == True:

    start_time = time.time()
    simulated_job = qmm.simulate(qua_config, my_program2, SimulationConfig(duration=int(10*1e3/4)))
    end_time = time.time()
    print(f"Simulation time: {end_time - start_time} s")

    # get DAC and digital samples
    samples = simulated_job.get_simulated_samples()
    plt.figure()
    plt.plot(samples.con1.analog["5"], "-")
    #plt.plot(samples.con1.analog["2"], "-")
    #plt.plot(samples.con1.analog["3"], "--")
    #plt.plot(samples.con1.digital["10"], "-")
    #plt.legend(("tweezer", "cooler", "repumper"))#,"trigger"))
    plt.xlabel("Time [ns]")
    plt.ylabel("Signal [V]")        
    plt.show()
else:
    qm = qmm.open_qm(qua_config)
    job = qm.execute(my_program)


2024-12-06 18:40:11,297 - qm - INFO     - Performing health check
2024-12-06 18:40:11,328 - qm - INFO     - Health check passed
2024-12-06 18:40:12,792 - qm - WARNING  - Open QM ended with warning 0: Opening a new Quantum Machine and closing Quantum Machines: 
2024-12-06 18:40:12,792 - qm - WARNING  - Open QM ended with warning 0: Quantum Machine qm-1733506781006 is canceling running job 1730223364785.
2024-12-06 18:40:12,806 - qm - INFO     - Sending program to QOP for compilation
2024-12-06 18:40:12,924 - qm - INFO     - Executing program


In [238]:
# halt the job
job.halt()

True

In [177]:
job.execution_report()

Execution report for job 1730223364732
Errors:
Please refer to section: Error Indications and Error Reporting in documentation for additional information

code		severity	message
10001		Error		Overflow error in analog output #6 of controller con1